# Crystal Representations

<div style="background-color: #f8d7da; border-left: 6px solid #ccc; margin: 20px; padding: 15px;">
    <strong>💡 John Stewart Bell:</strong> Theoretical physicists live in a classical world, looking out into a quantum-mechanical world.
</div>

<iframe class="speakerdeck-iframe" frameborder="0" src="https://speakerdeck.com/player/431daa62140b472bb2b19c499ebb73f5" title="Machine Learning for Materials (Lecture 4)" allowfullscreen="true" style="border: 0px; background-clip: padding-box; background-color: rgba(0, 0, 0, 0.1); margin: 0px; padding: 0px; border-radius: 6px; box-shadow: rgba(0, 0, 0, 0.2) 0px 5px 40px; width: 100%; height: auto; aspect-ratio: 560 / 420;" data-ratio="1.3333333333333333"></iframe>

[Lecture slides](https://speakerdeck.com/aronwalsh/mlformaterials-lecture4-data)

## 🚀 Navigating crystal space

Our world can be described as a three-dimensional space (or a four-dimensional spacetime continuum). Chemical space is even more vast when you think of all of the compositions and structures that can be built from combinations of the 118 known elements in the Periodic Table! Considering only carbon atoms, C-C bonds can be used to form molecules, rods, spheres, sheets, and extended crystals. 

Today's goal is to explore the combinatorial explosion that occurs for multi-component solids using informatics tools.

In [ ]:
# Installation of libraries
!pip install matminer --quiet
!pip install smact --quiet
!pip install plotly --quiet

In [ ]:
# Import of modules
import pandas as pd  # Data manipulation with DataFrames
import numpy as np  # Numerical operations
import matplotlib.pyplot as plt  # Plotting
import plotly.express as px  # More plotting
import pprint  # Pretty print data structures

<details>
<summary>Colab error solution</summary>
If running the import module cell fails with an "AttributeError", click `Runtime` -> `Restart Session` and then simply rerun the cell. 
</details>

## ABX<sub>3</sub> perovskite structure

Let's start by considering how we can represent a crystal structure in Python. We will make use of the functionality for storing and representing crystal structures in `pymatgen`. 

The [`Structure` module](https://pymatgen.org/pymatgen.core.structure.html) contains the `Structure` class which can be used to represent the unit cell of a crystal.  Remember, this is the simplest repeat unit of a crystal that is defined by three unit cell lengths (**a**,**b**,**c**), three angles ($\alpha,\beta,\gamma$), and the fractional coordinates of the atoms that it contains.

Below, we will create a model of a cubic perovskite from its spacegroup, Pm3̅m (No. 221). For a cubic unit cell **a** = **b** = **c** and $\alpha = \beta = \gamma = 90 ^{\circ}$, so we only need to define the length of **a**.

We will now generate a structure for CsPbI<sub>3</sub> from a spacegroup using the class method `Structure.from_spacegroup()`.

![image](./images/2_CsPbI3.png)

This graphic was created using [VESTA](https://jp-minerals.org/vesta/en).

In [ ]:
# Create a structure object for cubic CsPbI3 using pymatgen
from pymatgen.core import Structure, Lattice  # Structure modules

CsPbI3 = Structure.from_spacegroup(
'Pm-3m', # Spacegroup for a cubic perovskite
Lattice.cubic(6.41), # Cubic spacing of 6.41 Å
['Pb2+', 'Cs+', 'I-'], # Unique species of the ABX3 compound
[[0.,0.,0.],[0.5,0.5,0.5],[0.,0.,0.5]] # Fractional atomic coordinates of each site
)

# Print the structure details
print(CsPbI3)

Note that it has used the symmetry from the space group information to create a primitive unit cell with 5 atoms. Within `pymatgen`, we can export crystal structures to a variety of file formats using the `to` method of the `Structure` class. Some of these include the Crystallographic Information Format (cif) and POSCAR (the standard file format for the density functional theory code [VASP](https://www.vasp.at/wiki/index.php/Category:Theory)). A complete list of file formats can be found in [the docs](https://pymatgen.org/pymatgen.core.structure.html#pymatgen.core.structure.IStructure.to) for the `Structure` class.

Note, the `filename` argument of the `to` method needs to be specified to save the file to disk. Only supplying the `fmt` argument will result in a string being returned.

In [ ]:
# Print the file to screen
print('\nThe CIF format for CsPbI3 is shown below:\n')
print(CsPbI3.to(fmt='cif'))

# Export the structure to a CIF
# CsPbI3.to(filename='CsPbI3.cif')

# Download the CIF from Google Colab
# from google.colab import files
# files.download('CsPbI3.cif')

<details>
<summary> Code hint </summary>
You can export a .cif by uncommenting and running the lines above. These files can be opened in a visualiser of your choice (e.g. VESTA or Crystalmaker).
</details>

There are many useful functions for analysing these structures. As we are using Python, this analysis can be extended to large databases of thousands to millions of entries. Below we check distances and two methods for assigning coordination numbers (detailed [here](https://pubs.acs.org/doi/10.1021/acs.inorgchem.0c02996)).

In [ ]:
# Distance between the first and second atoms
distance = CsPbI3.get_distance(0, 1)
print(f"Distance between Pb and Cs: {distance:.2f} Å")

# Coordination environment of Pb using Voronoi tessellation
from pymatgen.analysis.local_env import VoronoiNN
vnn = VoronoiNN()
coord_env = vnn.get_nn(CsPbI3, 0)
print(f"\nCoordination number of Pb (VoronoiNN): {len(coord_env)}")

# Coordination environment of Pb using Crystal Near-Neighbor Algorithm 
from pymatgen.analysis.local_env import CrystalNN
crystal_nn = CrystalNN()
coord_env_crystal = crystal_nn.get_nn(CsPbI3, 0)
print(f"\nCoordination number of Pb (CrystalNN): {len(coord_env_crystal)}\n")
for site in coord_env_crystal:
    print(f"Distance to {site.species_string}: {CsPbI3.get_distance(0, site.index):.2f} Å")

# Assign oxidation states
from pymatgen.analysis.bond_valence import BVAnalyzer
analyzer = BVAnalyzer()
valences = analyzer.get_valences(CsPbI3)
print(f"\nAssigned oxidation states:")
for site, valence in zip(CsPbI3.sites, valences):
    print(f"{site.species_string}: {valence:+.1f}")

You can imagine how to hand-build simple feature vectors to describe crystal strutures, but we can do this more systematically using purpose-built representations for machine learning. Let's try to construct a Sine Coulomb matrix with `matminer`. 

In [ ]:
from matminer.featurizers.structure import SineCoulombMatrix

# Sine matrix representation
scm = SineCoulombMatrix(flatten=False)
scm.fit([CsPbI3])
scm_fea = scm.featurize(CsPbI3)
np.set_printoptions(precision=2, suppress=True) 
print("\nSine Coulomb matrix features:\n", np.array(scm_fea))

The output is a 5x5 matrix because there are 5 atoms in the unit cell of CsPbI<sub>3</sub>. Each element M<sub>ij</sub> in the matrix represents the interaction between atoms i and j.

- The **diagonal elements** (i.e., M<sub>ii</sub>) represent the interaction of each atom with itself, which depends on the atomic number Z<sub>i</sub> and is calculated as 0.5Z<sub>i</sub><sup>2.4</sup>.
- The **off-diagonal elements** (i.e., M<sub>ij</sub> for i ≠ j) are inspired by the Coulomb interaction between different atoms i and j, scaled by their atomic numbers Z<sub>i</sub> and Z<sub>j</sub>, and the distance between them in the periodic structure.

This matrix encodes information from both the positions and types of atoms, providing a feature vector. Let's see what happens when the chemistry changes.

In [ ]:
from pymatgen.core import Species

# Substitute Pb with Sn
CsSnI3 = CsPbI3.copy()
CsSnI3.replace_species({Species("Pb2+"): Species("Sn2+")})

# Sine matrix representation for CsSnI3 
scm = SineCoulombMatrix(flatten=False)
scm.fit([CsSnI3])
scm_fea_sn = scm.featurize(CsSnI3)

np.set_printoptions(precision=2, suppress=True) 
print("\nOriginal Coulomb matrix features:\n", np.array(scm_fea))
print("\nSine Coulomb matrix features with Sn replacing Pb:\n", np.array(scm_fea_sn))

<div style="background-color: #d4edda; border-left: 6px solid #ccc; margin: 20px; padding: 15px; border-radius: 5px;">
    <strong>🐢 Take a beat:</strong> Think about why some values remain the same, others are different, and where they are placed in the matrix.
</div>

While useful, this is not the state-of-the-art approach. One reason is that only pairwise information is included, e.g. two structures with different angles between the component atoms may have the same numerical representation. The [DScribe tutorials](https://singroup.github.io/dscribe/latest/tutorials/tutorials.html) contain a description of more advanced representations such as Smooth Overlap of Atomic Positions (SOAP) that includes both 2-body and 3-body descriptors of structure.

## Possible Perovskites

We can make an ionic approximation for CsPbI<sub>3</sub> and consider that the crystal is formed of Cs<sup>+</sup>, Pb<sup>2+</sup>, and I<sup>-</sup> ions. The crystal is charge neutral overall as the sum of cations (3<sup>+</sup>) is balanced by the sum of anions (3<sup>-</sup>). This charge balance is also what the structural analysis we performed suggested. It is worth remembering that bonding in real solids is more complicated than a simple classification of "ionic", "covalent" or "metallic" (e.g. see this [perspective](https://www.nature.com/articles/s41563-018-0165-7)).

Let's now screen a large space of possible Cs-Pb-I compounds by making use of our materials informatics code [smact](https://github.com/WMD-group/SMACT).

In [ ]:
import smact
from smact.screening import smact_filter
import pprint

# Define a list of three elements to screen for A-B-X compounds
elements = ["Cs", "Pb]

# Convert element symbols into smact.Element objects
element_objects = [smact.Element(e) for e in elements]

# Apply smact_filter to find valid compositions within a stoichiometry threshold
allowed_combinations = smact_filter(element_objects, threshold=8)

# Display the allowed combinations in a readable format
print(f"Number of allowed combinations: --> {len(allowed_combinations)} <--")
pprint.pprint("(elements), (charges), (stoichiometry)")
pprint.pprint(allowed_combinations)

<details>
<summary> Code hint </summary>
Close all quotations and add "I" to the elements list.
</details>

That set includes many possibilities including the regular perovskite stoichiometry CsPbI<sub>3</sub>, which has a 1:1:3 ratio, as well as many others such as Cs<sub>2</sub>Pb<sub></sub>I<sub>4</sub>, which forms a layered (2D octahedral connectivity) perovskite.  Some of the entries have Pb in a 4- charge state (known as the plumbide anion).

Before the exploration of metal halides, metals oxides were the most commonly studied perovskites. Let's explore how about A-B-O compounds can be formed.

In [ ]:
import itertools
import multiprocessing
from smact import Element, element_dictionary, ordered_elements
from smact.screening import smact_filter

# Load all elements and then limit to Bi
all_elements = element_dictionary()
elements = [all_elements[symbol] for symbol in ordered_elements(1, 83)]

# Generate all AB combinations + add oxygen (O)
metal_pairs = itertools.combinations(elements, 2)
ternary_systems = [[*pair, Element("O")] for pair in metal_pairs]
print(f"Raw element combinations (ABO): --> {len(ternary_systems)} <--")

# Apply chemical filters using multiprocessing
if __name__ == "__main__":
    with multiprocessing.Pool(processes=4) as pool:
        result = pool.map(smact_filter, ternary_systems)

# Flatten the list 
flat_list = [composition for sublist in result if sublist for composition in sublist]
print(f"Allowed stoichiometries: --> {len(flat_list)} <--")

# Print some valid compositions
print("  elements, oxidation states, stoichiometries")
for entry in flat_list[-10:]: 
    print(entry)

That's a lot of potential compounds. Let's filter them down to just keep ABO<sub>3</sub>.

In [ ]:
# Filter for (1,1,3) stoichiometries with oxide anion
filtered_compositions = [
    comp for comp in flat_list 
    if comp[2] == (1, 1, 3) 
    and 'O' in comp[0] 
    and comp[1][comp[0].index('O')] == -2  # Oxide
    and comp[1][0] > 0  # First element must have a positive charge
    and comp[1][1] > 0  # Second element must have a positive charge
]

print(f"Filtered (1,1,3) stoichiometries with O(-2): --> {len(filtered_compositions)} <--")

# Print a few filtered compositions
print("  elements, oxidation states, stoichiometries")
for entry in filtered_compositions[-10:]:  
    print(entry)

That's still too many to scroll through, but we can make a plot to visualise the combinations.

In [ ]:
# List of elements to include 
elements = sorted(set([el for comp in filtered_compositions for el in comp[0]]))

# DataFrame to hold the allowed combinations
matrix = pd.DataFrame(0, index=elements, columns=elements)

# Fill the matrix with allowed combinations
for comp in filtered_compositions:
    A, B, O = comp[0]
    matrix.at[A, B] = 1
    matrix.at[B, A] = 1  

# Create the plot 
fig = px.imshow(
    matrix.values,
    labels=dict(x="Element A", y="Element B", color="Allowed"),
    x=matrix.columns,
    y=matrix.index,
    color_continuous_scale="Plasma",
)

# Update layout for smaller font size
fig.update_layout(
    xaxis=dict(tickfont=dict(size=10)),
    yaxis=dict(tickfont=dict(size=10)),
    title="Charge allowed ABO<sub>3</sub> combinations",
    title_x=0.5,
)

fig.show()

## 🚨 Exercise 4

<div style="background-color: #dceefb; border-left: 6px solid #ccc; margin: 20px; padding: 15px; border-radius: 5px;">
    <strong>💡 Coding exercises:</strong> The exercises are designed to apply what you have learned with room for creativity. It is fine to discuss solutions with your classmates, but the actual code should not be directly copied.
</div>

### Your details

In [ ]:
import numpy as np

# Insert your values
Name = "No Name" # Replace with your name
CID = 123446 # Replace with your College ID (as a numeric value with no leading 0s)

# Set a random seed using the CID value
CID = int(CID)
np.random.seed(CID)

# Print the message
print("This is the work of " + Name + " [CID: " + str(CID) + "]")

### Problem

Materials informatics can harness the combinatorial spaces of chemical composition, crystals structure, and physical properties. A task will be given to explore another ternary space.

In [ ]:
#Empty block for your answers




In [ ]:
#Empty block for your answers




<div style="background-color: #d4edda; border-left: 6px solid #ccc; margin: 20px; padding: 15px; border-radius: 5px;">
    <strong>📓 Submission:</strong> When your notebook is complete in Google Colab, go to <em>File > Download</em> and choose <code>.ipynb</code>. The completed file should be uploaded to Blackboard under assignments for MATE70026.
</div>

## 🌊 Dive deeper

* _Level 1:_ Tackle Chapter 8 on Linear Unsupervised Learning in [Machine Learning Refined](https://github.com/jermwatt/machine_learning_refined#what-is-new-in-the-second-edition).

* _Level 2:_ Read about our attempt to screen _all inorganic materials_ (with caveats) in the journal [Chem](https://doi.org/10.1016/j.chempr.2016.09.010). 

* _Level 3:_ Watch a [seminar](https://www.youtube.com/watch?v=gd-uahI5xbA) by quantum chemist Anatole von Lilienfeld on chemical space. 